In [2]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
import re
from urlextract import URLExtract as extract
extractor = extract()

In [4]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# BFE PROD
#packages = 'https://ckan.opendata.swiss/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,a02f2157-1cfd-436e-909d-2e197a86f6bd,"{'fr': 'test_relations_01', 'de': 'test_relati...",https://www.bfe.admin.ch/bfe/de/home.html,{},"[{'byte_size': 0, 'coverage': '', 'cache_last_..."
1,44f9dd93-6bd0-4d56-8650-e472b44b72b0,{'fr': 'Offres d’encouragement de l’innovation...,http://www.bfe.admin.ch/ogd4,{},"[{'byte_size': 0, 'coverage': '', 'cache_last_..."
2,b4117aac-fc84-43de-8c0f-a74e0dd98247,"{'fr': '', 'de': '', 'en': 'test_relations', '...",https://www.bfe.admin.ch,[[{'url': 'https://www.bfe.admin.ch/bfe/de/hom...,"[{'byte_size': 0, 'coverage': '', 'cache_last_..."
3,75b319b5-a7bc-40fc-80e9-579b02e46c02,"{'fr': '', 'de': 'Kennzahlen Shared Mobility',...",https://bfe.admin.ch/shared-mobility-angebote,[[{'url': 'https://www.sharedmobility.ch/info'...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
4,20c76f59-1178-4357-8096-cf3bf164876e,{'fr': 'Réseaux thermiques: offre de chaleur o...,https://www.bfe.admin.ch/waerme-und-kaelteangebot,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Text (.csv), comma..."
...,...,...,...,...,...
130,ff382622-8419-4b66-a157-1e47b9653883,{'fr': 'Chiffres-clés concernant l’infrastruct...,http://www.uvek-gis.admin.ch/BFE/storymaps/MO_...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
131,a37cb9ae-50f9-4fa5-a48c-95e2c5f0625b,{'fr': 'Aptitude des toitures à utiliser l’éne...,https://www.bfe.admin.ch/bfe/de/home/versorgun...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
132,73d41982-f474-409a-a129-418678921128,{'fr': 'Etude sur l'électricité hivernale d'or...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 5819680686, 'cache_last_updated..."
133,e839fa78-ba22-4772-be16-0843b99b7161,{'fr': 'Résultats de la modélisation PE2050+ :...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.uvek-gis.admin.ch/BFE/st...,"[{'byte_size': 0, 'cache_last_updated': None, ..."


In [5]:
print(df['relations'].loc[2])

["[{'url': 'https://www.bfe.admin.ch/bfe/de/home.html', 'label': 'rel 1'}, {'url': 'https://www.bfe.admin.ch/bfe/de/home/forschung-und-cleantech/forschungsprogramme/bioenergie.html', 'label': 'rel 2'}, {'url': 'https://www.bfe.admin.ch/bfe/de/home/foerderung/energieeffizienz/gebaeudeprogramm.html', 'label': 'rel 3'}]"]


In [6]:
print(df['relations'].loc[4])

[{'url': 'https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/b6c8d9c6-a2ca-435a-af0a-6a8ac94199fc', 'label': 'geocat.ch permalink'}]
